# Masterboot image with Signature Provider for mcxn946

This notebook describes how to use a custom remote signing service for generating masterboot image using *nxpimage* tool.


In [1]:
%run ../../init_notebook.ipynb

import os
import pprint

pp = pprint.PrettyPrinter(indent=4)

plugins_dir = "../common/plugins/"

SASP_PLUGIN = os.path.join(plugins_dir, "sasp.py")
WORKSPACE = "workspace/mbimg/"  # change this to path to your workspace
DATA_DIR = "data_mbimg/"  # change this to path to your workspace
VERBOSITY = (
    "-v"  # verbosity of commands, might be -v or -vv for debug or blank for no additional info
)

env: JUPYTER_SPSDK=1
Created `%!` as an alias for `%execute`.


## Signature Provider Plugin

First, we need to setup the Signature Provider plugin and start the custom HSM. In order to do that, open the signature provider notebook (signature_provider.ipynb) and follow the instructions there. Once you are done, come back and continue here.

## Config File Setup

The masterboot configuration file will be needed for successful generation of Masterboot image(MBI) using *nxpimage* application.

There are three types of MBI for mcxn9xx based on the authentication type: `Plain`, `CRC` and `Signed`.

In this example we will be interested only in the `Signed` image type.

Run the following code and see how the generated configuration template looks like.

In [1]:
import os

from spsdk.utils.misc import load_file

# choose family for the MCU
FAMILY = "mcxn946"

%! nxpimage $VERBOSITY mbi get-templates --family $FAMILY --output $WORKSPACE --force

MBI_CONFIG_PATH = os.path.join(WORKSPACE, "mcxn946_xip_signed.yaml")
# just for verification that the template was generated
assert os.path.exists(MBI_CONFIG_PATH)

config_content = load_file(MBI_CONFIG_PATH)
print(config_content)

nxpimage -v mbi get-templates --family mcxn946 --output workspace/mbimg/ --force 
Creating workspace\mbimg\mcxn946_xip_plain.yaml template file.
Creating workspace\mbimg\mcxn946_xip_crc.yaml template file.
Creating workspace\mbimg\mcxn946_xip_signed.yaml template file.
Creating workspace\mbimg\mcxn946_load_to_ram_plain.yaml template file.
Creating workspace\mbimg\mcxn946_load_to_ram_crc.yaml template file.
# ===============  Master Boot Image Configuration template for mcxn946:latest, Plain Signed XIP Image.  ===============

# ======================================================================================================================
#                                                 == General Options ==                                                 
# ======================================================================================================================
# -------------------------------------===== The chip family name [Required] =====-----------------------

## Masterboot image generation

The generated configuration file contains all possible configuration settings. 

Some of them are required (labeled with `[Required]` comment), some of them are conditionally required (labeled with `[Conditionally required]` comment) and some are optional (labeled with `[Optional]` comment).

Let's now focus on two configuration settings: `signPrivateKey` and `signProvider`. These two configuration values are mutually exclusive, so only one can be chosen. 
In order to use Signature Provider, we will remove the line with `signPrivateKey`  and update the one with `signProvider`. 

Keep in mind that the signature provider configuration must meet following rules:
  - Configuration key
    - key names `sign_provider` or `signProvider` are allowed

  - Configuration value
    - format `"type=<identifier>;<key1>=<value1>;<key2>=<value2>;..."`
    - the `<identifier>` has to match the `identifier` class attribute defined in the custom signature provider(`plugins/sasp.py`)
    - the remaining key-value pairs are passed to the `__init__` method of the concrete Signature Provider
    - e.g.: `"type=file;file_path=private_key.pem"` will instantiate `spsdk.crypto.PlainFileSP(file_path='private_key.pem')`

For sake of simplification, the pre-generated master boot configuration file is used. It contains only settings related to this example.

In [1]:
import shutil

# Copy additional files needed for masterboot image creation
shutil.rmtree(WORKSPACE)
shutil.copytree(DATA_DIR, WORKSPACE)

CERT_BLOCK_CONFIG_PATH = os.path.join(WORKSPACE, "mcxn946_cert_block.yaml")

%! nxpimage cert-block export --family $FAMILY --config $CERT_BLOCK_CONFIG_PATH
pp.pprint(f"Cert block binary has been generated '{CERT_BLOCK_CONFIG_PATH}'")

%! nxpimage $VERBOSITY mbi export --plugin $SASP_PLUGIN --config $MBI_CONFIG_PATH
pp.pprint(f"Mbi binary has been generated '{os.path.join(WORKSPACE, 'my_mbi.bin')}'")

nxpimage cert-block export --family mcxn946 --config workspace/mbimg/mcxn946_cert_block.yaml "Cert block binary has been generated 'workspace/mbimg/mcxn946_cert_block.yaml'"

RKTH: b7ed92485e2090838d9d1766513de63a655c626700a7241dd2d6d72160051873ac9ad63a0da3a73dbfe3b2ddc652d31f
Success. (Certificate Block: workspace\mbimg\cert_block.bin created.)
nxpimage -v mbi export --plugin ../common/plugins/sasp.py --config workspace/mbimg/mcxn946_xip_signed.yaml "Mbi binary has been generated 'workspace/mbimg/my_mbi.bin'"

RKTH: b7ed92485e2090838d9d1766513de63a655c626700a7241dd2d6d72160051873ac9ad63a0da3a73dbfe3b2ddc652d31f
INFO:spsdk.apps.nxpimage:
+==0x0000_0000= Application Block ======+
|       Size: 13.9 kiB; 14,284 B        |
|+==0x0000_0000= Application ==========+|
||      Size: 13.6 kiB; 13,956 B       ||
|+==0x0000_3683========================+|
|+==0x0000_3684= Certification Block ==+|
||             Size: 208 B             ||
|+==0x0000_3753========================+|
|+==0x0000_3754= M

## Customer Manufacturing Programming Area (CMPA) generation

You can generate your own CMPA template and update it with your settings. 

In this example, we are using a pregenerated configuration file. Only non-default settings are kept in the config file. 

Pay special attention to the RoTK_USAGE register as it defines number of RoT keys in the certification block.  
In our case, we used 2 RoT keys, so the `RoTK2_Usage` and `RoTK3_Usage` are set to `KEY_SLOT_NOT_USED`.

In [1]:
CMPA_CONFIG_PATH = os.path.join(WORKSPACE, "mcxn946_cmpa.yaml")
pp.pprint(f"All config files are ready in folder '{WORKSPACE}'")

SF0 = os.path.join(WORKSPACE, "hsm_k0_secp384r1.pub")
SF1 = os.path.join(WORKSPACE, "hsm_k1_secp384r1.pub")
CMPA_BIN = os.path.join(WORKSPACE, "cmpa.bin")
%! pfr generate-binary -c $CMPA_CONFIG_PATH -sf $SF0 -sf $SF1 -o $CMPA_BIN
pp.pprint(f"CMPA has been generated in '{CMPA_BIN}'")

"All config files are ready in folder 'workspace/mbimg/'"
pfr generate-binary -c workspace/mbimg/mcxn946_cmpa.yaml -sf workspace/mbimg/hsm_k0_secp384r1.pub -sf workspace/mbimg/hsm_k1_secp384r1.pub -o workspace/mbimg/cmpa.bin 
Success. (PFR binary has been generated)
"CMPA has been generated in 'workspace/mbimg/cmpa.bin'"


## Execution

At this point, we have everything we need for running the application. Connect the board, update the `UART_CONNECTION` variable depending on the used serial port and run the code.
Keep in mind that the board must be in ISP mode. Once the code finishes, you can restart the board.
Once the application is booted, green LED starts blinking.

In [1]:
UART_CONNECTION = "-p com19"
%! blhost $UART_CONNECTION -- fill-memory 0x20000000 4 0xc0000405 word
%! blhost $UART_CONNECTION -- configure-memory 9 0x20000000
%! blhost $UART_CONNECTION -- flash-erase-region 0x80000000 0x10000
%! blhost $UART_CONNECTION -- fill-memory 0x20003000 4 0xF000000F word
%! blhost $UART_CONNECTION -- configure-memory 9 0x20003000
pp.pprint("Memory has been configured")

%! pfr erase-cmpa $UART_CONNECTION --family $FAMILY

CMPA_BIN = os.path.join(WORKSPACE, "cmpa.bin")
%! pfr write $UART_CONNECTION --type cmpa --family $FAMILY --binary $CMPA_BIN
pp.pprint(f"CMPA has been written'")

MBI_BIN = os.path.join(WORKSPACE, "my_mbi.bin")
%! blhost $UART_CONNECTION -- write-memory 0x80001000 $MBI_BIN

blhost -p com19 -- fill-memory 0x20000000 4 0xc0000405 word 
Response status = 0 (0x0) Success.
blhost -p com19 -- configure-memory 9 0x20000000 
Response status = 0 (0x0) Success.
blhost -p com19 -- flash-erase-region 0x80000000 0x10000 
Response status = 0 (0x0) Success.
blhost -p com19 -- fill-memory 0x20003000 4 0xF000000F word 
Response status = 0 (0x0) Success.
blhost -p com19 -- configure-memory 9 0x20003000 
Response status = 0 (0x0) Success.
'Memory has been configured'
pfr erase-cmpa -p com19 --family mcxn9xx 
CMPA page address on mcxn9xx is 0x1004000
CMPA page has been erased.
pfr write -p com19 --type cmpa --family mcxn9xx --binary workspace/mbimg/cmpa.bin 
CMPA page address on mcxn9xx is 0x1004000
CMPA data written to device.
"CMPA has been written'"
blhost -p com19 -- write-memory 0x80001000 workspace/mbimg/my_mbi.bin 
Writing memory
Response status = 0 (0x0) Success.
Response word 1 = 14284 (0x37cc)


## HSM teardown

Last step is to stop custom HSM. In order to do that, open again the HSM setup notebook (sahsm.ipynb) and stop the running jupyter notebook code cell.